<a href="https://colab.research.google.com/github/RaminTK/ML/blob/main/RTKDMV_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
import json
import time

# --- CONFIGURATION ---
TELEGRAM_BOT_TOKEN = 'xxxxxxx'
TELEGRAM_CHAT_ID = 'xxxxxxx'
URL = 'https://telegov.njportal.com/njmvc/AppointmentWizard/12'
CHECK_INTERVAL_MINUTES = 5

# --- TELEGRAM ALERT ---
def send_telegram(message):
    url = f'https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage'
    data = {'chat_id': TELEGRAM_CHAT_ID, 'text': message, 'parse_mode': 'Markdown'}
    response = requests.post(url, data=data)
    if response.status_code != 200:
        print("❌ Telegram send failed:", response.text)
    else:
        print("✅ Message sent!")


# --- NAMES ---

LOCATION_NAMES = {
    124: "Bakers Basin - Real ID",
    125: "Bayonne - Real ID",
    127: "Camden - Real ID",
    146: "Cardiff - Real ID",
    129: "Delanco - Real ID",
    130: "Eatontown - Real ID",
    132: "Edison - Real ID",
    265: "Elizabeth - Real ID",
    133: "Flemington - Real ID",
    135: "Freehold - Real ID",
    136: "Lodi - Real ID",
    511: "Manahawkin - Real ID",
    138: "Newark - Real ID",
    448: "Newton - Real ID",
    139: "North Bergen - Real ID",
    141: "Oakland - Real ID",
    142: "Paterson - Real ID",
    144: "Rahway - Real ID",
    145: "Randolph - Real ID",
    126: "Rio Grande - Real ID",
    502: "Runnemede - Real ID",
    128: "Salem - Real ID",
    131: "South Plainfield - Real ID",
    134: "Toms River - Real ID",
    137: "Vineland - Real ID",
    451: "Washington - Real ID",
    140: "Wayne - Real ID",
    143: "West Deptford - Real ID",
}

# --- TIME DATA CHECK ---
def check_time_data():
    response = requests.get(URL)
    html = response.text

    match = re.search(r'var\s+timeData\s*=\s*(\[\{.*?\}\])\s*;', html, re.DOTALL)
    if not match:
        # If above fails, fallback to greedy array capture until just before </script>
        match = re.search(r'var\s+timeData\s*=\s*(\[\{.*?\}])\s*[\r\n]', html, re.DOTALL)

    if not match:
        print("❌ Could not extract timeData.")
        send_telegram("⚠️ *DMV check failed:* timeData not found.")
        return

    raw_json = match.group(1).strip()

    try:
        # Remove any trailing \r\n or JS artifacts
        raw_json_clean = raw_json.split('\r\n')[0]
        time_data = json.loads(raw_json_clean)
    except json.JSONDecodeError as e:
        print("❌ JSON decode failed:", e)
        send_telegram(f"⚠️ *DMV check failed:* JSON decode error - {str(e)}.")
        return

    available = []
    for loc in time_data:
        slot_info = loc.get("FirstOpenSlot", "")
        if "Next Available:" in slot_info:
            location_id = loc.get("LocationId")
            next_time   = re.search(r'Next Available:\s*(.*)', slot_info).group(1).strip()
            loc_name    = LOCATION_NAMES.get(location_id, f"Location {location_id}")
            available.append(f"📍 *{loc_name}*\n🕓 *Next Available:* `{next_time}`")

    if available:
        message = "🚨 *Available DMV Appointments Found:*\n\n" + "\n\n".join(available)
    else:
        message = "ℹ️ *No DMV appointments available* at this time."

    send_telegram(message)

# --- LOOP ---
if __name__ == '__main__':
    while True:
        print("⏳ Checking DMV timeData availability...")
        check_time_data()
        print(f"🕒 Sleeping {CHECK_INTERVAL_MINUTES} minutes...\n")
        time.sleep(CHECK_INTERVAL_MINUTES * 60)
